### 과제

무신사 코디맵 1페이지(많은 댓글 수 기준 정렬)에 나와있는 60개의 코디맵에 대하여 각각

[’codimap_category’, ‘codimap_title’, ‘codimap_date’, ‘views’, ‘comment_numbers’] 정보를 가져와서 csv 형태로 만든 다음, 해당 코드(.ipynb)와 csv파일을 본인 github 과제 repository에 제출

https://www.musinsa.com/app/codimap/lists

In [3]:
pip install selenium==4.8.2

Note: you may need to restart the kernel to use updated packages.


In [4]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.8.2


In [5]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
codimap_list = []
url = 'https://store.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url)
driver.implicitly_wait(2)

interval = 1

#현재 문서 높이를 가져와서 저장
prev_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        
    # 페이지 로딩 대기
    time.sleep(interval)

    # 현재 문서 높이를 가져와서 저장
    curr_height = driver.execute_script("return document.body.scrollHeight")
    if curr_height == prev_height:
        break

    prev_height = curr_height


columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers']
values = []
soup = BeautifulSoup(driver.page_source, 'lxml')
data_rows = soup.find_all('li', attrs={'class': 'style-list-item'})

for i, row in enumerate(data_rows):

    print('{}번째 코디맵 크롤링'.format(i+1))
    blank = []
    
    codimap_category = row.find('span', attrs={'class': "style-list-information__text"})
    if codimap_category:
        codimap_category = codimap_category.get_text().strip()
        blank.append(codimap_category)
    else:
        blank.append('Something is wrong')
        print('{}번째 카테고리 가져올때 문제발생'.format(i+1))
        continue

    codimap_title = row.find('strong', attrs={'class': "style-list-information__title"})
    if codimap_title:
        codimap_title = codimap_title.get_text().strip()
        blank.append(codimap_title)
    else:
        blank.append('Something is wrong')
        print('{}번째 코디 제목 가져올때 문제발생'.format(i+1))
        continue

    codimap_date = row.find('span', attrs={'class': "post-information__text"})
    if codimap_date:
        codimap_date = codimap_date.get_text().strip()
        blank.append(codimap_date)
    else:
        blank.append('Something is wrong')
        print('{}번째 게시일 가져올때 문제발생'.format(i+1))
        continue

    views = row.select_one('span.post-information__text:nth-of-type(2)')
    if views:
        views = views.get_text().strip()[3:]
        blank.append(views)
    else:
        blank.append('Something is wrong')
        print('{}번째 조회 수 가져올때 문제발생'.format(i+1))
        continue

    comment_numbers = row.select_one('span.post-information__text:nth-of-type(3)')
    if comment_numbers:
        comment_numbers = comment_numbers.get_text().strip()[3:]
        blank.append(comment_numbers)
    else:
        blank.append('Something is wrong')
        print('{}번째 댓글 수 가져올때 문제발생'.format(i+1))
        continue

    values.append(blank)
    print('---------------------------------------------------')

df = pd.DataFrame(values, columns = columns)
codimap_list.append(df)

1번째 코디맵 크롤링
---------------------------------------------------
2번째 코디맵 크롤링
---------------------------------------------------
3번째 코디맵 크롤링
---------------------------------------------------
4번째 코디맵 크롤링
---------------------------------------------------
5번째 코디맵 크롤링
---------------------------------------------------
6번째 코디맵 크롤링
---------------------------------------------------
7번째 코디맵 크롤링
---------------------------------------------------
8번째 코디맵 크롤링
---------------------------------------------------
9번째 코디맵 크롤링
---------------------------------------------------
10번째 코디맵 크롤링
---------------------------------------------------
11번째 코디맵 크롤링
---------------------------------------------------
12번째 코디맵 크롤링
---------------------------------------------------
13번째 코디맵 크롤링
---------------------------------------------------
14번째 코디맵 크롤링
---------------------------------------------------
15번째 코디맵 크롤링
---------------------------------------------------
16번째 코디맵 크롤링
---------------------

In [7]:
df = pd.concat(codimap_list).reset_index(drop=True) # index 초기화 해줘야함
df.head(10)

,codimap_category,codimap_title,codimap_date,views,comment_numbers
0,캐주얼,톤온톤 코디,21.07.02,"16,057",49
1,캐주얼,블랙 매니아,21.02.23,"34,672",46
2,캐주얼,오늘의 코디,22.03.22,"12,200",38
3,댄디,추천 댄디 룩,21.09.15,"20,186",27
4,캐주얼,릴랙스 룩,21.02.23,"23,538",22
5,스트릿,힙하다 힙해,21.02.03,"35,288",21
6,캐주얼,심플한 코디,21.07.30,"39,577",18
7,댄디,데이트 코디,21.12.24,"10,311",18
8,댄디,훈훈 남친 룩,21.03.24,"19,345",16
9,캐주얼,지금 입기 좋아,21.08.31,"20,214",16


In [12]:
df.to_csv('크롤링 과제.csv', encoding='utf-8-sig')